## pandas

pandas是基于Numpy构建的，让以Numpy为中心的应用变得更加简单。

pandas引入约定：

In [1]:
from pandas import Series, DataFrame

In [2]:
import pandas as pd

### Series

Series是一种类似于一维数组的对象，它由一组数据以及索引组成，可以通过Series的values和index属性获取其数组表示形式和索引对象：

In [3]:
import numpy as np
obj = Series(np.arange(1,10,2))
obj

0    1
1    3
2    5
3    7
4    9
dtype: int32

In [4]:
obj.values

array([1, 3, 5, 7, 9])

In [5]:
obj.index

RangeIndex(start=0, stop=5, step=1)

Numpy数组运算（如根据布尔型数组进行过滤，标量乘法，应用数学函数等）都会保留索引和值之间的链接：

In [7]:
obj[obj%3==0]

1    3
4    9
dtype: int32

pandas的isnull和notnull函数可用于检测缺失数据：

In [8]:
pd.isnull(obj)

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [9]:
pd.notnull(obj)

0    True
1    True
2    True
3    True
4    True
dtype: bool

*pd.isnull(obj)等价于obj.isnull()*

Series对象本身及其索引都有一个name属性：

In [16]:
obj.name = 'obj'
obj.index.name = 'index'
obj

index
0    1
1    3
2    5
3    7
4    9
Name: obj, dtype: int32

### DataFrame

DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值，字符，布尔值等）。DataFrame既有行索引也有列索引，可以看作是由Series组成的字典（共用同一个索引）。

构建DataFrame的方法很多，最常用的是直接传入一个由等长列表或Numpy数组组成的字典：

In [11]:
data={
'year':[23,32,43,22],
'pop':np.arange(4)}
data

{'pop': array([0, 1, 2, 3]), 'year': [23, 32, 43, 22]}

In [12]:
frame=DataFrame(data)
frame

,pop,year
0,0,23
1,1,32
2,2,43
3,3,22


*如果指定离列序列，则DataFrame的列就会按照指定顺序进行排列，如：DataFrame（data,columns=['year','pop']*

通过类似字典标记的方式或属性的方式，可以将DataFrame的列获取为一个Series：

In [13]:
frame.year

0    23
1    32
2    43
3    22
Name: year, dtype: int64

为不存在的列赋值会创建出一个新列，如果赋值的是一个Series，就会精确匹配DataFrame的索引，空位将被填上缺失值：

In [14]:
val = Series([2,2,2],index=[0,2,4])
val

0    2
2    2
4    2
dtype: int64

In [15]:
frame['debt']

,pop,year,debt
0,0,23,2.0
1,1,32,NaN
2,2,43,2.0
3,3,22,NaN


关键字del用于删除列：

In [17]:
del frame['pop']
frame

,year,debt
0,23,2.0
1,32,NaN
2,43,2.0
3,22,NaN


嵌套字典（字典中的字典）：另一种常见的数据形式，将它传给DataFrame，它就会被解释为：外层字典的键作为列，内层键则作为行索引

In [18]:
pop = {'NNN':{2001:2.4,2002:2.9},
      'OOO':{2000:1.5,2001:1.8,2002:3.6}}
DataFrame(pop)

,NNN,OOO
2000,NaN,1.5
2001,2.4,1.8
2002,2.9,3.6


*当然也可以对结果进行转置.T，以及显式指定索引*

### 索引对象

pandas的索引对象负责管理轴标签和其它元数据(比如轴名称等）。构建Series和DataFrame时，所用到的任何数组或其它序列标签都会被转换成一个Index:

In [19]:
obj = Series(range(3),index=['a','b','c'])
obj.index

Index(['a', 'b', 'c'], dtype='object')

Index的对象是不可修改的，这样才能使Index对象在多个数据结构之间安全共享：

In [20]:
obj.index[1]='d'

TypeError: Index does not support mutable operations

### 基本功能
*一些操作Series和DataFram中的数据的基本手段*

**重新索引**：pandas对象的一个重要方法是reindex,其作用是创建一个适应新索引的新对象

In [21]:
obj = Series([1.1,2.2,3.3,4.4],index=['d','b','a','c'])

obj

d    1.1
b    2.2
a    3.3
c    4.4
dtype: float64

In [25]:
obj.reindex(['a','b','c','d','e','f'])

a    3.3
b    2.2
c    4.4
d    1.1
e    NaN
f    NaN
dtype: float64

In [27]:
#reindex(index,method,column)
#缺失值填充为0
obj.reindex(['a','b','c','d','e','f'],fill_value=0)

a    3.3
b    2.2
c    4.4
d    1.1
e    0.0
f    0.0
dtype: float64

In [30]:
obj2 = Series(['blue','purple','yellow'],index=[0,2,4])
#ffill实现前向值填充
obj2.reindex(range(6),method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

*ffill&pad 前向 bfill&backfill 后向
插值只能按轴0应用*

**标签索引**：利用ix可以使重新索引任务变得更简洁

In [33]:
frame.ix[range(4),['debt','year']]

,debt,year
0,2.0,23
1,NaN,32
2,2.0,43
3,NaN,22


**丢弃指定轴上的项**：drop方法返回的是一个在指定轴上删除了指定值的新对象

In [3]:
from pandas import Series,DataFrame
import numpy as np

data = DataFrame(np.arange(16).reshape((4,4)),
                index=['A','B','C','D'],
                columns=['one','two','three','four'])

data.drop(['two','four'],axis=1)

,one,three
A,0,2
B,4,6
C,8,10
D,12,14


In [4]:
data.drop('A')

,one,two,three,four
B,4,5,6,7
C,8,9,10,11
D,12,13,14,15


**索引，选取和过滤**：利用标签的切片运算与普通的Python切片运算不同，其末端是包含的

In [5]:
data['B':'C']

,one,two,three,four
B,4,5,6,7
C,8,9,10,11


In [6]:
data[['three','one']]

,three,one
A,2,0
B,6,4
C,10,8
D,14,12


按行列索引:

In [12]:
data.ix[data['three']>5,:3]

,one,two,three
B,4,5,6
C,8,9,10
D,12,13,14


**算术运算和数据对齐**：pandas可以对不同的索引对象进行算术运算，对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集，并在不重叠的索引处引入NA值（缺失值会在算术运算过程中传播）

In [13]:
s1=Series(np.arange(4),index=['a','b','c','d'])
s2=Series(np.arange(4),index=['a','d','e','f'])

s1+s2

a    0.0
b    NaN
c    NaN
d    4.0
e    NaN
f    NaN
dtype: float64

缺失值的填充

In [14]:
df1=DataFrame(np.arange(12.).reshape((3,4)),columns=list('abcd'))
df2=DataFrame(np.arange(16.).reshape((4,4)),columns=list('abce'))

df1+df2

,a,b,c,d,e
0,0.0,2.0,4.0,NaN,NaN
1,8.0,10.0,12.0,NaN,NaN
2,16.0,18.0,20.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN


In [15]:
#使用add方法，传入加数以及一个fill_value参数
#在列上广播必须使用算术运算方法
df1.add(df2,fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,3.0,3.0
1,8.0,10.0,12.0,7.0,7.0
2,16.0,18.0,20.0,11.0,11.0
3,12.0,13.0,14.0,NaN,15.0


In [16]:
df1.sub(df1['d'],axis=0)

,a,b,c,d
0,-3.0,-2.0,-1.0,0.0
1,-3.0,-2.0,-1.0,0.0
2,-3.0,-2.0,-1.0,0.0


***
add 加法(+)
sub 减法(-)
div 除法(/)
mul 乘法(*)
***

**函数的应用和映射**：Numpy的ufuncs（元素级数组方法）也可以用于惭怍pandas对象，另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。DataFrame的apply方法即可实现此功能

In [18]:
f=lambda x:x.max()-x.min()

df1.apply(f,axis=1)

0    3.0
1    3.0
2    3.0
dtype: float64

此外，元素级的Python函数也是可以用的，假如你想得到df1中各个浮点值的格式化字符串，使用applymap即可：

In [19]:
fmat = lambda x:'%.2f'%x

df1.applymap(fmat)

,a,b,c,d
0,0.00,1.00,2.00,3.00
1,4.00,5.00,6.00,7.00
2,8.00,9.00,10.00,11.00


之所以叫做applymap,是因为Series有一个用于应用元素级函数的map方法：

In [20]:
df1['c'].map(fmat)

0     2.00
1     6.00
2    10.00
Name: c, dtype: object

**排序和排名**：要对行或列索引进行排序（按字典顺序），可使用sort_index方法，它将返回一个已排序的新对象

In [34]:
obj=DataFrame(np.arange(8).reshape((4,2)),index=list('dacb'),columns=['three','one'])

obj.sort_index(ascending=False).sort_index(axis=1)

,one,three
d,1,0
c,5,4
b,7,6
a,3,2


排序函数Series.sort()以及Series.order()都已经被弃用了，使用新的Series.sort_values()来替代，任何缺失值默认都会被放到Series的末尾

In [30]:
obj.three.sort_values()

d    0
a    2
c    4
b    6
Name: three, dtype: int32

DataFrame中对多列值进行排序，将一个或多个列的名字传递给by选项即可达到该目的，对索引排序使用sort_index()函数对值排序使用sort_values()：

In [32]:
frame=DataFrame({'b':[4,7,-3,2],'a':[0,1,0,1]})

frame.sort_values(by=['a','b'])

,a,b
2,0,-3
0,0,4
3,1,2
1,1,7


排名（ranking）会增设一个排名值（从1开始，一直到数组中有效数据的数量），rank是通过“为各组分配一个平均排名”的方式破坏平级关系的:

In [36]:
obj=Series([7,-5,7,4,2,0,4])
#Series.rank(axis=0, method='average',ascending=True)
#method:average,min,max,first
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

***
rank()将源值从小到大的分配从1开始的值,
若遇上n个相同值，则分配的值=值的排名+(n-1)*0.5
***

**带有重复值的轴索引**：对于带有重复值的索引，数据选取的行为将会有些不同。如果某个索引对应多个值，则返回一个Series；而对应单个值的，则返回一个标量值。

In [37]:
obj=Series(range(5),index=['a','a','b','b','c'])
#索引的is_unique属性可以告诉你它的值是否是唯一的
obj.index.is_unique

False

In [38]:
obj['a']

a    0
a    1
dtype: int32

### 汇总和计算描述统计

调用DataFrame的统计方法时，NA值会自动被排除，除非整个行或列都是NA，通过skipna选项可以禁用该功能。

![统计](assets/markdown-img-paste-20170726191713120.png)

![统计2](assets/markdown-img-paste-20170726191911810.png)

**唯一值、值计数以及成员资格**：unique可以得到Series中的唯一值数组，value_counts用于计算数组或序列中各值出现的频率，isin用于判断矢量化集合的成员资格。

In [5]:
obj=Series(list('cadaabbcc'))

obj.value_counts().sort_index()

a    3
b    2
c    3
d    1
dtype: int64

In [6]:
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [7]:
obj[obj.isin(['b','c'])]

0    c
5    b
6    b
7    c
8    c
dtype: object

### 处理缺失数据

![NA](assets/markdown-img-paste-20170726194943303.png)

**滤除缺失数据**：对于一个Series，dropna返回一个仅含非空数据和索引值的Series

In [8]:
from numpy import nan as NA

data=Series([1,NA,3.5,NA,7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

*也可以用布尔型索引达到这个目的：data[data.notnull()]*

对于DataFrame对象，dropna默认丢弃任何含有缺失值的行，传入how='all'将只丢弃全为NA的那些行：dropna(how='all')

**填充缺失数据**：通过一个常数调用fillna就会将缺失值替代为那个常数值，也可以通过调用fillna，以实现对不同行或列填充不同的值

In [9]:
data.fillna({1:10,3:30})

0     1.0
1    10.0
2     3.5
3    30.0
4     7.0
dtype: float64

对reindex有效的那些插值方法也可以用于fillna

In [11]:
data.fillna(method='ffill')

0    1.0
1    1.0
2    3.5
3    3.5
4    7.0
dtype: float64

![na0](assets/markdown-img-paste-20170726201548570.png)

![na1](assets/markdown-img-paste-20170726201646188.png)

### 层次化索引

层次化索引使你能在一个轴上拥有多个索引级别

In [15]:
data=Series(np.random.randn(10),
           index=[['a','a','a','b','b','b','c','c','d','d'],
                 [1,2,3,1,2,3,1,2,2,3]])
data

a  1    1.137381
   2   -0.320811
   3   -0.488938
b  1   -1.374554
   2   -0.656628
   3   -0.821464
c  1    0.212299
   2   -1.762240
d  2   -0.583129
   3   -3.633820
dtype: float64

In [16]:
#索引
data['b']

1   -1.374554
2   -0.656628
3   -0.821464
dtype: float64

In [17]:
#在内层中进行选取
data[:,2]

a   -0.320811
b   -0.656628
c   -1.762240
d   -0.583129
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。比如说，这段数据可以通过unstack方法被重新安排到一个DataFrame中：

In [18]:
data.unstack()

,1,2,3
a,1.137381,-0.320811,-0.488938
b,-1.374554,-0.656628,-0.821464
c,0.212299,-1.762240,NaN
d,NaN,-0.583129,-3.633820


*unstack的逆运算是stack()*

**根据级别汇总统计**：可以根据行或列上的级别来进行求和

In [21]:
#各层都可以有名字（字符串，其它Python对象），索引名称与轴标签的区别？
data.index.names=['key1','key2']

data

key1  key2
a     1       1.137381
      2      -0.320811
      3      -0.488938
b     1      -1.374554
      2      -0.656628
      3      -0.821464
c     1       0.212299
      2      -1.762240
d     2      -0.583129
      3      -3.633820
dtype: float64

In [22]:
#level选项用于指定在某条轴上求和的级别，轴1：axis=1
data.sum(level='key2')

key2
1   -0.024873
2   -3.322808
3   -4.944222
dtype: float64

*其实是利用了pandas的groupby功能*

**使用DataFrame的列**：set_index()函数会将其一个或多个列转换为行索引，默认情况下哪些列会从DataFrame中移除，使用drop=False就可将其保留下来。reset_index()的功能跟set_index()刚好相反，层次化索引的级别会被转移到列里面：

In [23]:
data.reset_index()

,key1,key2,0
0,a,1,1.137381
1,a,2,-0.320811
2,a,3,-0.488938
3,b,1,-1.374554
4,b,2,-0.656628
5,b,3,-0.821464
6,c,1,0.212299
7,c,2,-1.762240
8,d,2,-0.583129
9,d,3,-3.633820
